# Utility

In [5]:
import os

def add_suffix_to_filename(file_path, suffix):
  """
  파일 경로의 확장자를 유지하면서 파일명에 접미사를 추가합니다.

  Args:
    file_path (str): 원본 파일 경로
    suffix (str): 파일명에 추가할 문자열

  Returns:
    str: 파일명이 변경된 새로운 파일 경로
  """
  # 1. 파일 경로를 파일명(루트)과 확장자로 분리
  # 예: "/path/to/document.txt" -> ("/path/to/document", ".txt")
  name_without_ext, extension = os.path.splitext(file_path)

  # 2. 파일명에 원하는 문자열 추가
  new_name_without_ext = name_without_ext + suffix

  # 3. 새로운 파일명과 확장자를 다시 합침
  new_file_path = new_name_without_ext + extension
  
  return new_file_path

# --- 사용 예시 ---
original_path = "/Users/test/documents/report.pdf"
suffix_to_add = "_final"

new_path = add_suffix_to_filename(original_path, suffix_to_add)

print(new_path)

/Users/test/documents/report_final.pdf


# Matrix Handler

## Matrix 추출, CSV 변환

### 함수정의부

In [13]:
import pandas as pd
import os

def convert_and_rename_xlsx_to_csv(folder_path):
    """
    XLSX 파일을 읽어 AOI 컬럼명을 고유하게 변경한 후 CSV로 저장합니다.
    """
    # CSV 파일들을 저장할 하위 폴더 생성
    csv_output_folder = os.path.join(folder_path, "csv_files_unique_names")
    os.makedirs(csv_output_folder, exist_ok=True)
    
    all_files = os.listdir(folder_path)
    xlsx_files = [f for f in all_files if f.endswith('.xlsx')]

    # 모든 파일에 공통으로 존재하는 기준 컬럼들
    base_cols = [
        'ProbeName', 'ProbeDisplayName', 'TargetName', 'HUGOSymbol', 
        'Accessions', 'GenomeBuild', 'GenomicPosition', 'AnalyteType', 
        'CodeClass', 'ProbePool', 'TargetGroup', 'GeneID'
    ]

    print(f"총 {len(xlsx_files)}개의 XLSX 파일을 CSV로 변환 및 컬럼명 변경 작업을 시작합니다...")
    for i, file in enumerate(xlsx_files):
        try:
            file_path = os.path.join(folder_path, file)
            df = pd.read_excel(file_path, sheet_name="BioProbeCountMatrix", engine='openpyxl')
            
            # 파일명을 기반으로 접두사(prefix) 생성 (확장자 제외)
            file_prefix = os.path.splitext(file)[0]
            
            # AOI 컬럼들 (base_cols에 속하지 않는 모든 컬럼)
            aoi_cols = [col for col in df.columns if col not in base_cols]
            
            # 새로운 컬럼명을 담을 딕셔너리 생성
            # 예: {'기존AOI_1': '파일명_기존AOI_1', '기존AOI_2': '파일명_기존AOI_2'}
            rename_dict = {col: f"{file_prefix}_{col}" for col in aoi_cols}
            
            # 컬럼명 변경 적용
            df.rename(columns=rename_dict, inplace=True)
            
            # CSV로 저장
            csv_filename = f"{file_prefix}.csv"
            df.to_csv(os.path.join(csv_output_folder, csv_filename), index=False)
            
            print(f"({i+1}/{len(xlsx_files)}) '{file}' -> '{csv_filename}' 변환 완료.")
        except Exception as e:
            print(f"'{file}' 처리 중 오류: {e}")
            
    print(f"\n✅ 작업 완료! 고유한 컬럼명을 가진 CSV 파일들이 '{csv_output_folder}' 폴더에 저장되었습니다.")



### Claude style

#### utilities and configurations

In [ ]:
import pandas as pd
import os
import warnings
from openpyxl import load_workbook
# 하위 폴더 이름 설정
subfolder_name="csv_files_1"

# warning 억제
warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')

# 설정값들
BASE_COLUMNS = [
    'ProbeName', 'ProbeDisplayName', 'TargetName', 'HUGOSymbol', 
    'Accessions', 'GenomeBuild', 'GenomicPosition', 'AnalyteType', 
    'CodeClass', 'ProbePool', 'TargetGroup', 'GeneID'
]

def get_excel_files(folder_path):
    """폴더에서 xlsx 파일들 찾기"""
    all_files = os.listdir(folder_path)
    return [f for f in all_files if f.endswith('.xlsx')]

def create_output_folder(folder_path):
    """CSV 파일 저장할 폴더 생성"""
    csv_output_folder = os.path.join(folder_path, subfolder_name)
    os.makedirs(csv_output_folder, exist_ok=True)
    return csv_output_folder

def has_target_sheet(file_path, sheet_name="BioProbeCountMatrix"):
    """시트 존재 여부를 빠르게 확인"""
    try:
        wb = load_workbook(file_path, read_only=True, data_only=True)
        result = sheet_name in wb.sheetnames
        wb.close()
        return result
    except Exception:
        return False

def process_single_file(file_path, file_prefix):
    """단일 파일 처리하여 DataFrame 반환"""
    # 파일 읽기 (read_only=True로 속도 향상)
    df = pd.read_excel(file_path, sheet_name="BioProbeCountMatrix", engine='openpyxl')
    
    # AOI 컬럼들 찾기 (기본 컬럼이 아닌 것들)
    aoi_cols = [col for col in df.columns if col not in BASE_COLUMNS]
    
    # 컬럼명 변경 (파일명_컬럼명)
    rename_dict = {col: f"{file_prefix}_{col}" for col in aoi_cols}
    df.rename(columns=rename_dict, inplace=True)
    
    return df, len(aoi_cols)

#### main function

In [ ]:
def convert_xlsx_to_csv(folder_path):
    """XLSX 파일들을 CSV로 변환하는 메인 함수"""
    
    # Excel 파일들 찾기
    xlsx_files = get_excel_files(folder_path)
    if not xlsx_files:
        print("❌ XLSX 파일을 찾을 수 없습니다!")
        return
    
    # 출력 폴더 생성
    csv_output_folder = create_output_folder(folder_path)
    
    print(f"📁 총 {len(xlsx_files)}개의 XLSX 파일을 처리합니다...")
    
    # 각 파일 처리
    success_count = 0
    skip_count = 0
    
    for i, file in enumerate(xlsx_files):
        file_path = os.path.join(folder_path, file)
        file_prefix = os.path.splitext(file)[0]  # 확장자 제거
        
        # 먼저 시트 존재 여부 빠르게 확인
        if not has_target_sheet(file_path):
            skip_count += 1
            print(f"⏭️  ({i+1}/{len(xlsx_files)}) {file} - BioProbeCountMatrix 시트 없음, 건너뛰기")
            continue
        
        try:
            # 파일 처리
            df, aoi_count = process_single_file(file_path, file_prefix)
            
            # CSV로 저장
            csv_filename = f"{file_prefix}.csv"
            df.to_csv(os.path.join(csv_output_folder, csv_filename), index=False)
            
            success_count += 1
            print(f"✅ ({i+1}/{len(xlsx_files)}) {file} → {csv_filename} (AOI 컬럼 {aoi_count}개)")
            
        except Exception as e:
            print(f"❌ ({i+1}/{len(xlsx_files)}) {file} 처리 실패: {e}")
    
    print(f"\n🎉 완료! {success_count}개 변환, {skip_count}개 건너뛰기 (총 {len(xlsx_files)}개)")
    print(f"   결과는 '{csv_output_folder}' 폴더에 저장되었습니다.")

### 변수정의

In [14]:
target_folder="/data/kjc1/projects/#999.GeoMxmeta/pooled"

### 실행

In [15]:
convert_and_rename_xlsx_to_csv(target_folder)
# Claude style


총 114개의 XLSX 파일을 CSV로 변환 및 컬럼명 변경 작업을 시작합니다...


/home/jaecheon/miniconda3/envs/scenvi/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


(1/114) 'DKD_20240125.xlsx' -> 'DKD_20240125.csv' 변환 완료.
't4p.xlsx' 처리 중 오류: Worksheet named 'BioProbeCountMatrix' not found


/home/jaecheon/miniconda3/envs/scenvi/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


(3/114) '#124_mIBD_biologics_95AOI.xlsx' -> '#124_mIBD_biologics_95AOI.csv' 변환 완료.


...

(112/114) 'TOF_geomx_analysis.xlsx' -> 'TOF_geomx_analysis.csv' 변환 완료.


/home/jaecheon/miniconda3/envs/scenvi/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


(113/114) 'Wound_GeoMx.xlsx' -> 'Wound_GeoMx.csv' 변환 완료.


/home/jaecheon/miniconda3/envs/scenvi/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


(114/114) 'Wound_KHW.xlsx' -> 'Wound_KHW.csv' 변환 완료.

✅ 작업 완료! 고유한 컬럼명을 가진 CSV 파일들이 '/data/kjc1/projects/#999. GeoMx meta/pooled/csv_files_unique_names' 폴더에 저장되었습니다.


## Matrix Merger

### Merger 함수정의부

In [ ]:
import pandas as pd
import os
import numpy as np
from functools import reduce
from typing import Dict, List, Tuple, Optional
import warnings
warnings.filterwarnings('ignore')

class OptimizedDataMerger:
    """
    최적화된 데이터 병합 클래스
    - 파일 유형 자동 감지 (CTA, Human WTA, Mouse)
    - 메모리 효율적인 처리
    - 병렬 처리 가능한 구조
    """
    
    def __init__(self, csv_folder_path: str, temp_export: bool = False):
        self.csv_folder_path = csv_folder_path
        self.temp_export = temp_export
        self.output_folder = os.path.join(csv_folder_path, "merged")
        
        # 표준 메타데이터 컬럼
        self.standard_cols = [
            'ProbeName', 'ProbeDisplayName', 'TargetName', 'HUGOSymbol', 
            'Accessions', 'GenomeBuild', 'GenomicPosition', 'AnalyteType', 
            'CodeClass', 'ProbePool', 'TargetGroup', 'GeneID'
        ]
        
        # 컬럼명 정규화 맵
        self.normalization_map = {
            col.lower().replace(' ', '').replace('_', ''): col 
            for col in self.standard_cols
        }
        
        # 파일 유형별 데이터 컨테이너
        self.cta_data = {'aoi_dfs': [], 'meta_map': {}}
        self.human_wta_data = {'aoi_dfs': [], 'meta_map': {}}
        self.mouse_data = {'aoi_dfs': [], 'meta_map': {}}
        
    def detect_file_type(self, df: pd.DataFrame) -> str:
        """
        파일 유형을 자동으로 감지
        Returns: 'CTA', 'HUMAN_WTA', 'MOUSE', or None
        """
        # GenomeBuild로 먼저 Mouse 확인
        if 'GenomeBuild' in df.columns:
            genome_builds = df['GenomeBuild'].astype(str).str.lower()
            
            if genome_builds.str.contains('grcm', na=False).any():
                return 'MOUSE'
            
            # Human 데이터인 경우 CTA vs WTA 구분
            if genome_builds.str.contains('grch', na=False).any():
                # Negative Probe의 TargetName으로 구분
                if 'CodeClass' in df.columns and 'TargetName' in df.columns:
                    neg_probes = df[df['CodeClass'] == 'Negative']
                    if not neg_probes.empty:
                        target_names = neg_probes['TargetName'].unique()
                        if 'Negative Probe' in target_names:
                            return 'CTA'
                        elif 'NegProbe-WTX' in target_names:
                            return 'HUMAN_WTA'
                
                # Negative Probe가 없는 경우, 같은 Gene에 여러 Probe가 있는지 확인
                if 'GeneID' in df.columns and 'TargetName' in df.columns:
                    endogenous = df[df.get('CodeClass', '') == 'Endogenous']
                    if not endogenous.empty:
                        gene_probe_counts = endogenous.groupby(['GeneID', 'TargetName'])['ProbeName'].nunique()
                        if (gene_probe_counts > 1).any():
                            return 'CTA'
                        else:
                            return 'HUMAN_WTA'
        
        return None
    
    def optimize_dtypes(self, df: pd.DataFrame) -> pd.DataFrame:
        """데이터 타입 최적화로 메모리 사용량 감소"""
        for col in df.columns:
            if col == 'ProbeName':  # ProbeName은 문자열로 유지
                continue
                
            col_type = df[col].dtype
            
            if col_type != 'object':
                try:
                    # NaN이 아닌 값만 선택
                    non_null = df[col].dropna()
                    
                    if len(non_null) == 0:  # 모든 값이 NaN인 경우
                        continue
                    
                    c_min = non_null.min()
                    c_max = non_null.max()
                    
                    # Integer 최적화 (NaN이 있으면 float로 유지)
                    if str(col_type).startswith('int') and not df[col].isna().any():
                        if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                            df[col] = df[col].astype(np.int8)
                        elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                            df[col] = df[col].astype(np.int16)
                        elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                            df[col] = df[col].astype(np.int32)
                    
                    # Float 최적화
                    elif str(col_type).startswith('float'):
                        # float16은 정밀도 문제가 있을 수 있으므로 float32로 제한
                        if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                            df[col] = df[col].astype(np.float32)
                            
                except Exception as e:
                    # 최적화 실패 시 원본 유지
                    continue
        
        return df
    
    def process_file(self, filepath: str, filename: str) -> None:
        """개별 파일 처리"""
        try:
            # 청크 단위로 읽어서 메모리 효율성 향상
            df = pd.read_csv(filepath, low_memory=False, chunksize=None)
        except Exception as e:
            print(f"  ❌ 파일 읽기 오류: {e}")
            return
        
        # 컬럼명 정규화
        df.rename(columns={
            col: self.normalization_map.get(
                col.lower().replace(' ', '').replace('_', ''), col
            ) for col in df.columns
        }, inplace=True)
        
        # 중복 컬럼 제거 (rename 후 발생 가능)
        df = df.loc[:, ~df.columns.duplicated()]
        
        if 'ProbeName' not in df.columns:
            print(f"  ⚠️  '{filename}'에 ProbeName 컬럼이 없어 건너뜁니다.")
            return
        
        # ProbeName을 문자열로 통일
        df['ProbeName'] = df['ProbeName'].astype(str)
        
        # 파일 유형 감지
        file_type = self.detect_file_type(df)
        if not file_type:
            print(f"  ⚠️  '{filename}'의 파일 유형을 확인할 수 없습니다.")
            return
        
        print(f"  📁 파일 유형: {file_type}")
        print(f"  📊 데이터 크기: {df.shape[0]} rows × {df.shape[1]} columns")
        
        # 데이터 타입 최적화
        df = self.optimize_dtypes(df)
        
        # 파일 유형에 따라 적절한 컨테이너에 저장
        if file_type == 'CTA':
            target_data = self.cta_data
        elif file_type == 'HUMAN_WTA':
            target_data = self.human_wta_data
        else:  # MOUSE
            target_data = self.mouse_data
        
        # AOI 컬럼 추출
        numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
        aoi_cols = [col for col in numeric_cols if col not in self.standard_cols]
        
        if aoi_cols:
            # 파일명 접두사
            file_prefix = os.path.splitext(filename)[0]
            
            # AOI 데이터프레임 생성
            aoi_df = df[['ProbeName'] + aoi_cols].copy()
            
            # 컬럼명에 파일 접두사 추가
            rename_dict = {col: f"{file_prefix}_{col}" for col in aoi_cols}
            aoi_df.rename(columns=rename_dict, inplace=True)
            
            # AOI 데이터 저장
            target_data['aoi_dfs'].append(aoi_df)
            
            # 메타데이터 수집 (중복 제거)
            meta_cols_to_use = ['ProbeName'] + [col for col in self.standard_cols if col in df.columns and col != 'ProbeName']
            # 중복 컬럼 제거
            meta_cols_to_use = list(dict.fromkeys(meta_cols_to_use))
            
            meta_df = df[meta_cols_to_use].drop_duplicates(subset=['ProbeName'])
            
            for idx, row in meta_df.iterrows():
                # ProbeName을 스칼라 값으로 확실히 변환
                try:
                    probe_name = str(row['ProbeName']) if not isinstance(row['ProbeName'], str) else row['ProbeName']
                except Exception as e:
                    print(f"    ⚠️  ProbeName 변환 오류 (row {idx}): {e}")
                    continue
                    
                if probe_name not in target_data['meta_map']:
                    target_data['meta_map'][probe_name] = {
                        col: row.get(col) for col in self.standard_cols 
                        if col in df.columns and col != 'ProbeName'  # ProbeName은 키로 사용하므로 제외
                    }
        
        print(f"  ✅ 처리 완료: {len(aoi_cols)}개 AOI 컬럼")
    
    def merge_and_save(self, data_dict: dict, output_name: str) -> Optional[pd.DataFrame]:
        """데이터 병합 및 저장"""
        if not data_dict['aoi_dfs']:
            print(f"[{output_name}] 처리할 데이터가 없습니다.")
            return None
        
        print(f"\n[{output_name}] 병합 시작...")
        
        try:
            # 모든 데이터프레임의 ProbeName을 문자열로 통일
            for df in data_dict['aoi_dfs']:
                df['ProbeName'] = df['ProbeName'].astype(str)
            
            # 효율적인 병합: 작은 데이터프레임부터 병합
            data_dict['aoi_dfs'].sort(key=lambda x: len(x))
            
            # AOI 데이터 병합
            merged_aoi = reduce(
                lambda left, right: pd.merge(left, right, on='ProbeName', how='outer'),
                data_dict['aoi_dfs']
            )
            
            # 메타데이터 데이터프레임 생성
            if data_dict['meta_map']:
                meta_df = pd.DataFrame.from_dict(data_dict['meta_map'], orient='index')
                
                # index를 ProbeName 컬럼으로 변환
                # 이미 ProbeName 컬럼이 있다면 제거
                if 'ProbeName' in meta_df.columns:
                    meta_df = meta_df.drop(columns=['ProbeName'])
                
                meta_df.index.name = 'ProbeName'
                meta_df.reset_index(inplace=True)
                meta_df['ProbeName'] = meta_df['ProbeName'].astype(str)
                
                # 최종 병합
                final_df = pd.merge(meta_df, merged_aoi, on='ProbeName', how='right')
            else:
                # 메타데이터가 없는 경우
                final_df = merged_aoi
            
            # 컬럼 순서 정리
            meta_cols = [col for col in self.standard_cols if col in final_df.columns]
            data_cols = [col for col in final_df.columns if col not in meta_cols]
            final_df = final_df[meta_cols + data_cols]
            
            # 데이터 타입 최적화
            final_df = self.optimize_dtypes(final_df)
            
            # 파일 저장
            output_path = self.get_unique_filepath(f"merged_{output_name}.csv")
            final_df.to_csv(output_path, index=False)
            
            print(f"  ✅ 저장 완료: {output_path}")
            print(f"     크기: {final_df.shape[0]} rows × {final_df.shape[1]} columns")
            print(f"     메모리 사용량: {final_df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
            
            return final_df
            
        except Exception as e:
            print(f"  ❌ 병합 중 오류 발생: {e}")
            import traceback
            traceback.print_exc()
            return None
    
    def get_unique_filepath(self, filename: str) -> str:
        """중복되지 않는 파일명 생성"""
        base, ext = os.path.splitext(filename)
        counter = 1
        filepath = os.path.join(self.output_folder, filename)
        
        while os.path.exists(filepath):
            filepath = os.path.join(self.output_folder, f"{base}_{counter}{ext}")
            counter += 1
        
        return filepath
    
    def run(self) -> Dict[str, pd.DataFrame]:
        """전체 처리 프로세스 실행"""
        # 폴더 확인 및 생성
        try:
            all_files = os.listdir(self.csv_folder_path)
            csv_files = [
                f for f in all_files 
                if f.endswith('.csv') and not f.startswith('merged_')
            ]
            
            if not csv_files:
                print(f"'{self.csv_folder_path}' 폴더에 처리할 CSV 파일이 없습니다.")
                return {}
        
        except FileNotFoundError:
            print(f"오류: '{self.csv_folder_path}' 폴더를 찾을 수 없습니다.")
            return {}
        
        # 출력 폴더 생성
        os.makedirs(self.output_folder, exist_ok=True)
        
        print(f"🔄 총 {len(csv_files)}개 CSV 파일 처리 시작\n")
        print("="*60)
        
        # 각 파일 처리
        for i, filename in enumerate(csv_files, 1):
            print(f"\n[{i}/{len(csv_files)}] 처리 중: {filename}")
            filepath = os.path.join(self.csv_folder_path, filename)
            self.process_file(filepath, filename)
        
        print("\n" + "="*60)
        print("📊 최종 병합 및 저장")
        print("="*60)
        
        # 결과 저장
        results = {}
        
        # CTA 데이터 병합 및 저장
        if self.cta_data['aoi_dfs']:
            print(f"\n📝 CTA 파일 {len(self.cta_data['aoi_dfs'])}개 발견")
            result = self.merge_and_save(self.cta_data, 'CTA')
            if result is not None:
                results['CTA'] = result
        
        # Human WTA 데이터 병합 및 저장  
        if self.human_wta_data['aoi_dfs']:
            print(f"\n📝 Human WTA 파일 {len(self.human_wta_data['aoi_dfs'])}개 발견")
            result = self.merge_and_save(self.human_wta_data, 'HUMAN_WTA')
            if result is not None:
                results['HUMAN_WTA'] = result
        
        # Mouse 데이터 병합 및 저장
        if self.mouse_data['aoi_dfs']:
            print(f"\n📝 Mouse 파일 {len(self.mouse_data['aoi_dfs'])}개 발견")
            result = self.merge_and_save(self.mouse_data, 'MOUSE')
            if result is not None:
                results['MOUSE'] = result
        
        print("\n" + "="*60)
        print("✨ 모든 처리가 완료되었습니다!")
        print(f"📁 결과 파일 위치: {self.output_folder}")
        
        return results


def process_and_merge_data_optimized(csv_folder_path: str, temp_export: bool = False) -> Dict[str, pd.DataFrame]:
    """
    최적화된 데이터 병합 함수
    
    Parameters:
    -----------
    csv_folder_path : str
        CSV 파일들이 있는 폴더 경로
    temp_export : bool
        중간 파일 저장 여부 (현재 미사용)
    
    Returns:
    --------
    Dict[str, pd.DataFrame]
        파일 유형별 병합된 데이터프레임 딕셔너리
    """
    merger = OptimizedDataMerger(csv_folder_path, temp_export)
    return merger.run()


# # 사용 예시
# if __name__ == "__main__":
#     # 폴더 경로 설정
#     folder_path = "./data"  # 실제 경로로 변경하세요
    
#     try:
#         # 처리 실행
#         results = process_and_merge_data_optimized(folder_path, temp_export=False)
        
#         # 결과 확인
#         for file_type, df in results.items():
#             if df is not None:
#                 print(f"\n{file_type} 데이터:")
#                 print(f"  - Shape: {df.shape}")
#                 print(f"  - ProbeName 유니크 개수: {df['ProbeName'].nunique()}")
#                 if 'CodeClass' in df.columns:
#                     print(f"  - CodeClass 분포:")
#                     print(f"    {df['CodeClass'].value_counts().to_dict()}")
                    
#     except Exception as e:
#         print(f"\n❌ 처리 중 오류 발생: {e}")
#         import traceback
#         traceback.print_exc()

### 변수 정의

In [ ]:
csv_source_folder="/data/kjc1/projects/#999.GeoMxmeta/pooled/csv_files/"

### 실행

In [ ]:
# 사용 예시
if __name__ == "__main__":
    # 폴더 경로 설정
    folder_path = csv_source_folder  # 실제 경로로 변경하세요
    
    try:
        # 처리 실행
        results = process_and_merge_data_optimized(folder_path, temp_export=False)
        
        # 결과 확인
        for file_type, df in results.items():
            if df is not None:
                print(f"\n{file_type} 데이터:")
                print(f"  - Shape: {df.shape}")
                print(f"  - ProbeName 유니크 개수: {df['ProbeName'].nunique()}")
                if 'CodeClass' in df.columns:
                    print(f"  - CodeClass 분포:")
                    print(f"    {df['CodeClass'].value_counts().to_dict()}")
                    
    except Exception as e:
        print(f"\n❌ 처리 중 오류 발생: {e}")
        import traceback
        traceback.print_exc()

## Filter

### Filter 함수정의부

In [1]:
import pandas as pd
import numpy as np
import hashlib
from typing import Dict, List, Tuple, Optional, Union
import json
from pathlib import Path

class DuplicateAOIFilter:
    """
    중복 AOI(Area of Interest) 컬럼을 감지하고 필터링하는 클래스
    완전히 동일한 데이터를 가진 AOI 컬럼들을 찾아서 제거
    """
    
    def __init__(self, verbose: bool = True):
        """
        Parameters:
        -----------
        verbose : bool
            상세한 처리 과정 출력 여부
        """
        self.verbose = verbose
        self.duplicate_groups = {}
        self.removal_report = {}
        
    def generate_column_fingerprint(self, series: pd.Series) -> str:
        """
        컬럼의 고유한 fingerprint 생성
        NaN 값도 고려하여 동일한 패턴을 가진 컬럼들을 식별
        
        Parameters:
        -----------
        series : pd.Series
            fingerprint를 생성할 데이터 시리즈
            
        Returns:
        --------
        str : 해시된 fingerprint
        """
        # NaN 위치 정보 포함
        nan_mask = series.isna()
        
        # NaN이 아닌 값들을 정렬하지 않고 그대로 사용 (순서 보존)
        # 값들을 문자열로 변환하여 해시 생성
        values_str = '|'.join([
            f"{i}:{v}" if not pd.isna(v) else f"{i}:NaN" 
            for i, v in enumerate(series.values)
        ])
        
        # MD5 해시 생성 (충돌 가능성은 매우 낮음)
        hash_object = hashlib.md5(values_str.encode())
        return hash_object.hexdigest()
    
    def find_duplicate_aois(self, df: pd.DataFrame) -> Dict[str, List[str]]:
        """
        중복된 AOI 컬럼들을 찾아서 그룹화
        
        Parameters:
        -----------
        df : pd.DataFrame
            분석할 데이터프레임
            
        Returns:
        --------
        Dict[str, List[str]] : fingerprint별 중복 컬럼 리스트
        """
        # 메타데이터 컬럼 식별
        meta_columns = ['ProbeName', 'ProbeDisplayName', 'TargetName', 'HUGOSymbol', 
                       'Accessions', 'GenomeBuild', 'GenomicPosition', 'AnalyteType', 
                       'CodeClass', 'ProbePool', 'TargetGroup', 'GeneID']
        
        # AOI 컬럼만 추출 (메타데이터 제외)
        aoi_columns = [col for col in df.columns if col not in meta_columns]
        
        if self.verbose:
            print(f"📊 전체 컬럼 수: {len(df.columns)}")
            print(f"📊 AOI 컬럼 수: {len(aoi_columns)}")
            print(f"🔍 중복 검사 시작...\n")
        
        # 각 AOI 컬럼의 fingerprint 생성
        fingerprint_map = {}
        
        for col in aoi_columns:
            fp = self.generate_column_fingerprint(df[col])
            
            if fp not in fingerprint_map:
                fingerprint_map[fp] = []
            fingerprint_map[fp].append(col)
        
        # 중복 그룹만 추출 (2개 이상의 컬럼을 가진 그룹)
        duplicate_groups = {
            fp: cols for fp, cols in fingerprint_map.items() 
            if len(cols) > 1
        }
        
        self.duplicate_groups = duplicate_groups
        
        if self.verbose:
            print(f"🔍 발견된 중복 그룹: {len(duplicate_groups)}개")
            total_duplicates = sum(len(cols) - 1 for cols in duplicate_groups.values())
            print(f"📉 제거 가능한 중복 AOI: {total_duplicates}개\n")
            
            # 중복 그룹 상세 정보 출력
            for i, (fp, cols) in enumerate(duplicate_groups.items(), 1):
                print(f"  그룹 {i}: {len(cols)}개 동일한 AOI")
                print(f"    대표 AOI: {cols[0]}")
                print(f"    중복 AOI: {', '.join(cols[1:])}")
                print()
        
        return duplicate_groups
    
    def filter_duplicates(self, 
                          df: pd.DataFrame, 
                          keep_strategy: str = 'first',
                          custom_priority: Optional[List[str]] = None) -> pd.DataFrame:
        """
        중복 AOI를 제거한 데이터프레임 반환
        
        Parameters:
        -----------
        df : pd.DataFrame
            필터링할 데이터프레임
        keep_strategy : str
            유지할 컬럼 선택 전략
            - 'first': 각 그룹에서 첫 번째 컬럼 유지 (기본값)
            - 'last': 각 그룹에서 마지막 컬럼 유지
            - 'shortest': 가장 짧은 이름의 컬럼 유지
            - 'custom': custom_priority 리스트 기준으로 우선순위 결정
        custom_priority : List[str]
            keep_strategy='custom'일 때 사용할 우선순위 패턴
            예: ['slide1', 'slide2'] - slide1이 있으면 우선 선택
            
        Returns:
        --------
        pd.DataFrame : 중복이 제거된 데이터프레임
        """
        # 먼저 중복 찾기
        duplicate_groups = self.find_duplicate_aois(df)
        
        if not duplicate_groups:
            if self.verbose:
                print("✅ 중복된 AOI가 없습니다.")
            return df.copy()
        
        # 제거할 컬럼 리스트
        columns_to_remove = []
        self.removal_report = {}
        
        for fp, cols in duplicate_groups.items():
            if keep_strategy == 'first':
                keep_col = cols[0]
                remove_cols = cols[1:]
            elif keep_strategy == 'last':
                keep_col = cols[-1]
                remove_cols = cols[:-1]
            elif keep_strategy == 'shortest':
                keep_col = min(cols, key=len)
                remove_cols = [c for c in cols if c != keep_col]
            elif keep_strategy == 'custom' and custom_priority:
                # 우선순위에 따라 선택
                keep_col = cols[0]  # 기본값
                for pattern in custom_priority:
                    matching = [c for c in cols if pattern in c]
                    if matching:
                        keep_col = matching[0]
                        break
                remove_cols = [c for c in cols if c != keep_col]
            else:
                keep_col = cols[0]
                remove_cols = cols[1:]
            
            columns_to_remove.extend(remove_cols)
            self.removal_report[keep_col] = remove_cols
        
        # 중복 제거된 데이터프레임 생성
        filtered_df = df.drop(columns=columns_to_remove)
        
        if self.verbose:
            print(f"\n✂️  제거된 컬럼 수: {len(columns_to_remove)}")
            print(f"✅ 남은 전체 컬럼 수: {len(filtered_df.columns)}")
            print(f"💾 메모리 절감: {(1 - filtered_df.memory_usage(deep=True).sum() / df.memory_usage(deep=True).sum()) * 100:.1f}%")
        
        return filtered_df
    
    def get_removal_report(self) -> Dict[str, List[str]]:
        """
        제거 리포트 반환
        
        Returns:
        --------
        Dict[str, List[str]] : 유지된 컬럼과 제거된 컬럼들의 매핑
        """
        return self.removal_report
    
    def save_report(self, filepath: str = "duplicate_aoi_report.json"):
        """
        중복 제거 리포트를 JSON 파일로 저장
        
        Parameters:
        -----------
        filepath : str
            저장할 파일 경로
        """
        report = {
            'summary': {
                'total_duplicate_groups': len(self.duplicate_groups),
                'total_removed_columns': sum(len(v) for v in self.removal_report.values()),
                'kept_columns': list(self.removal_report.keys())
            },
            'details': self.removal_report,
            'duplicate_groups': {
                f"group_{i}": cols 
                for i, cols in enumerate(self.duplicate_groups.values(), 1)
            }
        }
        
        with open(filepath, 'w') as f:
            json.dump(report, f, indent=2)
        
        if self.verbose:
            print(f"\n📄 리포트 저장 완료: {filepath}")
    
    def validate_filtering(self, original_df: pd.DataFrame, filtered_df: pd.DataFrame) -> bool:
        """
        필터링 결과 검증
        중요한 데이터가 손실되지 않았는지 확인
        
        Parameters:
        -----------
        original_df : pd.DataFrame
            원본 데이터프레임
        filtered_df : pd.DataFrame
            필터링된 데이터프레임
            
        Returns:
        --------
        bool : 검증 성공 여부
        """
        # 메타데이터 컬럼은 모두 유지되었는지 확인
        meta_columns = ['ProbeName', 'ProbeDisplayName', 'TargetName', 'HUGOSymbol', 
                       'Accessions', 'GenomeBuild', 'GenomicPosition', 'AnalyteType', 
                       'CodeClass', 'ProbePool', 'TargetGroup', 'GeneID']
        
        original_meta = [col for col in meta_columns if col in original_df.columns]
        filtered_meta = [col for col in meta_columns if col in filtered_df.columns]
        
        if original_meta != filtered_meta:
            print("⚠️ 경고: 메타데이터 컬럼이 변경되었습니다!")
            return False
        
        # 행 수가 동일한지 확인
        if len(original_df) != len(filtered_df):
            print("⚠️ 경고: 행 수가 변경되었습니다!")
            return False
        
        if self.verbose:
            print("\n✅ 검증 완료: 데이터 무결성 확인")
        
        return True


def remove_duplicate_aois(filepath: Union[str, pd.DataFrame], 
                         output_path: Optional[str] = None,
                         keep_strategy: str = 'first',
                         verbose: bool = True) -> pd.DataFrame:
    """
    중복 AOI를 제거하는 간단한 wrapper 함수
    
    Parameters:
    -----------
    filepath : Union[str, pd.DataFrame]
        CSV 파일 경로 또는 데이터프레임
    output_path : Optional[str]
        결과를 저장할 CSV 파일 경로 (None이면 저장하지 않음)
    keep_strategy : str
        유지할 컬럼 선택 전략 ('first', 'last', 'shortest')
    verbose : bool
        상세 출력 여부
        
    Returns:
    --------
    pd.DataFrame : 중복이 제거된 데이터프레임
    
    Example:
    --------
    >>> # 파일 경로로 사용
    >>> filtered_df = remove_duplicate_aois('merged_CTA.csv', 
    ...                                     output_path='merged_CTA_filtered.csv')
    
    >>> # 데이터프레임으로 사용
    >>> df = pd.read_csv('merged_CTA.csv')
    >>> filtered_df = remove_duplicate_aois(df)
    """
    # 입력 처리
    if isinstance(filepath, str):
        if verbose:
            print(f"📁 파일 읽기: {filepath}")
        df = pd.read_csv(filepath, low_memory=False)
    else:
        df = filepath.copy()
    
    # 필터 객체 생성 및 처리
    filter_obj = DuplicateAOIFilter(verbose=verbose)
    filtered_df = filter_obj.filter_duplicates(df, keep_strategy=keep_strategy)
    
    # 검증
    filter_obj.validate_filtering(df, filtered_df)
    
    # 리포트 저장
    if output_path:
        # 리포트는 같은 디렉토리에 저장
        report_path = Path(output_path).parent / f"{Path(output_path).stem}_duplicate_report.json"
        filter_obj.save_report(str(report_path))
    
    # 결과 저장
    if output_path:
        filtered_df.to_csv(output_path, index=False)
        if verbose:
            print(f"💾 필터링된 파일 저장: {output_path}")
            print(f"   원본: {df.shape[1]} columns → 필터링: {filtered_df.shape[1]} columns")
    
    return filtered_df


def analyze_aoi_similarity(df: pd.DataFrame, threshold: float = 0.99) -> pd.DataFrame:
    """
    AOI 간의 유사도를 분석 (완전 동일하지 않지만 매우 유사한 경우 찾기)
    
    Parameters:
    -----------
    df : pd.DataFrame
        분석할 데이터프레임
    threshold : float
        유사도 임계값 (0-1, 기본값 0.99 = 99% 유사)
        
    Returns:
    --------
    pd.DataFrame : 유사한 AOI 쌍과 유사도 점수
    """
    # 메타데이터 컬럼 제외
    meta_columns = ['ProbeName', 'ProbeDisplayName', 'TargetName', 'HUGOSymbol', 
                   'Accessions', 'GenomeBuild', 'GenomicPosition', 'AnalyteType', 
                   'CodeClass', 'ProbePool', 'TargetGroup', 'GeneID']
    
    aoi_columns = [col for col in df.columns if col not in meta_columns]
    
    similar_pairs = []
    
    print(f"🔍 {len(aoi_columns)}개 AOI 컬럼 간 유사도 분석 중...")
    
    for i, col1 in enumerate(aoi_columns):
        for col2 in aoi_columns[i+1:]:
            # 두 컬럼의 상관계수 계산
            corr = df[col1].corr(df[col2])
            
            if corr >= threshold:
                # 값이 정확히 일치하는 비율 계산
                exact_match_ratio = (df[col1] == df[col2]).mean()
                
                similar_pairs.append({
                    'Column1': col1,
                    'Column2': col2,
                    'Correlation': corr,
                    'ExactMatchRatio': exact_match_ratio,
                    'IsDuplicate': exact_match_ratio == 1.0
                })
    
    result_df = pd.DataFrame(similar_pairs)
    
    if len(result_df) > 0:
        result_df = result_df.sort_values('Correlation', ascending=False)
        print(f"✅ {len(result_df)}개의 유사한 AOI 쌍 발견 (threshold: {threshold})")
        
        duplicates = result_df[result_df['IsDuplicate'] == True]
        if len(duplicates) > 0:
            print(f"   - 완전 중복: {len(duplicates)}쌍")
        
        high_similar = result_df[(result_df['IsDuplicate'] == False) & (result_df['Correlation'] >= threshold)]
        if len(high_similar) > 0:
            print(f"   - 높은 유사도: {len(high_similar)}쌍")
    else:
        print(f"ℹ️ 임계값 {threshold} 이상의 유사한 AOI 쌍이 없습니다.")
    
    return result_df


# 사용 예시
if __name__ == "__main__":
    # 기본 사용법
    print("=" * 60)
    print("중복 AOI 필터링 예시")
    print("=" * 60)
    
    # 1. 간단한 사용법
    # filtered_df = remove_duplicate_aois('merged_CTA.csv', 
    #                                     output_path='merged_CTA_filtered.csv')
    
    # 2. 상세 분석과 함께 사용
    # filter_obj = DuplicateAOIFilter(verbose=True)
    # df = pd.read_csv('merged_CTA.csv')
    # duplicate_groups = filter_obj.find_duplicate_aois(df)
    # filtered_df = filter_obj.filter_duplicates(df, keep_strategy='shortest')
    # filter_obj.save_report('cta_duplicate_report.json')
    
    # 3. 유사도 분석
    # df = pd.read_csv('merged_CTA.csv')
    # similarity_df = analyze_aoi_similarity(df, threshold=0.95)
    # print(similarity_df.head())
    
    print("\n사용 방법:")
    print("1. remove_duplicate_aois('merged_CTA.csv') - 간단 사용")
    print("2. DuplicateAOIFilter 클래스 - 상세 제어")
    print("3. analyze_aoi_similarity() - 유사도 분석")

중복 AOI 필터링 예시

사용 방법:
1. remove_duplicate_aois('merged_CTA.csv') - 간단 사용
2. DuplicateAOIFilter 클래스 - 상세 제어
3. analyze_aoi_similarity() - 유사도 분석


### 변수 정의

In [10]:
raw_merged_csv='/data/kjc1/projects/#999.GeoMxmeta/pooled/csv_files/merged/merged_HUMAN_WTA.csv'
output_path=add_suffix_to_filename(raw_merged_csv,"_filtered")
raw_merged_csv2='/data/kjc1/projects/#999.GeoMxmeta/pooled/csv_files/merged/merged_CTA.csv'
output_path2=add_suffix_to_filename(raw_merged_csv2,"_filtered")
raw_merged_csv3='/data/kjc1/projects/#999.GeoMxmeta/pooled/csv_files/merged/merged_MOUSE.csv'
output_path3=add_suffix_to_filename(raw_merged_csv3,"_filtered")
print(output_path)
print(output_path2)
print(output_path3)

/data/kjc1/projects/#999. GeoMx meta/pooled/csv_files/merged/merged_HUMAN_WTA_filtered.csv
/data/kjc1/projects/#999. GeoMx meta/pooled/csv_files/merged/merged_CTA_filtered.csv
/data/kjc1/projects/#999. GeoMx meta/pooled/csv_files/merged/merged_MOUSE_filtered.csv


### 실행

In [9]:
# 사용 예시
if __name__ == "__main__":
    # 기본 사용법
    print("=" * 60)
    print("중복 AOI 필터링 예시")
    print("=" * 60)
    
    # 1. 간단한 사용법
    filtered_df = remove_duplicate_aois(raw_merged_csv, 
                                        output_path=output_path)
    
    # 2. 상세 분석과 함께 사용
    # filter_obj = DuplicateAOIFilter(verbose=True)
    # df = pd.read_csv('merged_CTA.csv')
    # duplicate_groups = filter_obj.find_duplicate_aois(df)
    # filtered_df = filter_obj.filter_duplicates(df, keep_strategy='shortest')
    # filter_obj.save_report('cta_duplicate_report.json')
    
    # 3. 유사도 분석
    # df = pd.read_csv('merged_CTA.csv')
    # similarity_df = analyze_aoi_similarity(df, threshold=0.95)
    # print(similarity_df.head())
    
    print("\n사용 방법:")
    print("1. remove_duplicate_aois('merged_CTA.csv') - 간단 사용")
    print("2. DuplicateAOIFilter 클래스 - 상세 제어")
    print("3. analyze_aoi_similarity() - 유사도 분석")

중복 AOI 필터링 예시
📁 파일 읽기: /data/kjc1/projects/#999. GeoMx meta/pooled/csv_files/merged/merged_HUMAN_WTA.csv
📊 전체 컬럼 수: 5469
📊 AOI 컬럼 수: 5457
🔍 중복 검사 시작...

🔍 발견된 중복 그룹: 1137개
📉 제거 가능한 중복 AOI: 1643개

  그룹 1: 137개 동일한 AOI
    대표 AOI: (not aligned)20221207 bk,yk melanoma_melanoma | 001 | Segment 1
    중복 AOI: (not aligned)20221207 bk,yk melanoma_melanoma | 001 | Segment 2, (not aligned)20221207 bk,yk melanoma_melanoma | 002 | Segment 1, (not aligned)20221207 bk,yk melanoma_melanoma | 002 | Segment 2, (not aligned)20221207 bk,yk melanoma_melanoma | 003 | Segment 1, (not aligned)20221207 bk,yk melanoma_melanoma | 003 | Segment 2, (not aligned)20221207 bk,yk melanoma_melanoma | 004 | Segment 1, (not aligned)20221207 bk,yk melanoma_melanoma | 004 | Segment 2, (not aligned)20221207 bk,yk melanoma_melanoma | 005 | Segment 1, (not aligned)20221207 bk,yk melanoma_melanoma | 005 | Segment 2, (not aligned)20221207 bk,yk melanoma_melanoma | 006 | Segment 1, (not aligned)20221207 bk,yk melanoma_melanoma

In [12]:
filtered_df = remove_duplicate_aois(raw_merged_csv,output_path=output_path)
filtered_df2 = remove_duplicate_aois(raw_merged_csv2,output_path=output_path2)
filtered_df3 = remove_duplicate_aois(raw_merged_csv3,output_path=output_path3)

📁 파일 읽기: /data/kjc1/projects/#999. GeoMx meta/pooled/csv_files/merged/merged_CTA.csv
📊 전체 컬럼 수: 85
📊 AOI 컬럼 수: 73
🔍 중복 검사 시작...

🔍 발견된 중복 그룹: 21개
📉 제거 가능한 중복 AOI: 34개

  그룹 1: 2개 동일한 AOI
    대표 AOI: CRC(2)_20211007 slide1 | 001 | Macrophage
    중복 AOI: CRC_20211007 slide1 | 001 | Macrophage

  그룹 2: 4개 동일한 AOI
    대표 AOI: CRC(2)_20211007 slide1 | 002 | Macrophage
    중복 AOI: CRC(2)_20211007 slide1 | 006 | Macrophage, CRC_20211007 slide1 | 002 | Macrophage, CRC_20211007 slide1 | 006 | Macrophage

  그룹 3: 2개 동일한 AOI
    대표 AOI: CRC(2)_20211007 slide1 | 003 | Macrophage
    중복 AOI: CRC_20211007 slide1 | 003 | Macrophage

  그룹 4: 2개 동일한 AOI
    대표 AOI: CRC(2)_20211007 slide1 | 004 | Macrophage
    중복 AOI: CRC_20211007 slide1 | 004 | Macrophage

  그룹 5: 2개 동일한 AOI
    대표 AOI: CRC(2)_20211007 slide1 | 005 | Macrophage
    중복 AOI: CRC_20211007 slide1 | 005 | Macrophage

  그룹 6: 2개 동일한 AOI
    대표 AOI: CRC(2)_20211007 slide1 | 007 | Macrophage
    중복 AOI: CRC_20211007 slide1 | 007 | Macrophage


# Metadata Handler

## Metadata 추출, csv 변환

### 오류!

함수 설명

GeomXProcessor 하에..
<!-- detect_data_type: columns를 다 합쳐서, "LOT_Cancer_Transcriptome_Atlas"가 있으면 CTA, "LOT_Mouse_NGS"가 있으면 mouse, 아니면 WTA로 분류
extract_segment_properties: openpyxl로 xlsx 읽어서 SegmentProperties sheet 찾아 pa로 출력
analyze_unique_columns:(AUC) WTA/CTA/mouse에 따라 common columns를 제한 unique columns 얻어내기. 데이터타입 알아내고, 고유값 샘플 (최대 10개) 얻어냄. unique_columns_analysis에 저장. 전체 데이터 얻어내 files_data에 저장.
infer_column_type: 데이터 타입 알아내는 함수. AUC내에서 사용됨. true, false, yes, no, 1, 0, t, f, y, n이 사용되면 boolean으로 간주. Empty, Boolean, Numeric, Category(가짓수가 AOI수의 10% 미만), String
process_directory: 디렉토리 내 모든 xlsx파일 찾아 처리
save_analysis_results: 결과를 csv파일로 출력
generate_mapping_template: column 매핑 템플릿 생성; 즉, dataset 1, 2, 3에서 사용된 PanCK, Panck, ck 같은 서로 다른 column명들을 어떻게 바꿔줄지 (예: "CK"), 값들은 뭘로 바꿔줄지 (예: True->TRUE)
apply_mappings_and_merge: 위에서 만든 매핑 파일을 읽어서, 모든 메타데이터를 변환시키고, 병합. -->
AOIMatcher 하에...
<!-- match_aoi_columns: merged matrix & merged metadata를 읽어서 매칭시킴.
매칭 딕셔너리를 생성한다. metadata의 AOI명에 대하여, 1) 이것이 matrix의 AOI명에 있으면 그냥 매칭시키고, 2) 없으면, pattern은 _{}, | {}, {}이 _로 분리된 것의 마지막에 있는지, |로 분리된 것의 마지막에 있는지로 찾음.
# -> 이거 좀 많이 잘못된거같은데..? 오히려 맨 앞에 나와야 하는데.
Matrix에만 있는 column를 정리해둠. 매칭안된 것들. -->

### ver1. 함수정의부; .py 문서로 사용되어야 함.

In [ ]:
#!/usr/bin/env python3
"""
GeomX Data Processor - Metadata Column Harmonization Tool
리눅스 서버 환경용 Python CLI 버전
"""

import pandas as pd
import numpy as np
import os
import sys
import argparse
import hashlib
import json
from pathlib import Path
from typing import Dict, List, Tuple, Optional
import warnings
warnings.filterwarnings('ignore')

# 공통 columns 정의 (다양한 데이터 타입별로)
COMMON_COLUMNS = {
    'WTA': [
        'SlideName', 'ScanLabel', 'ROILabel', 'SegmentLabel', 'SegmentDisplayName',
        'Origin Instrument ID', 'QCFlags', 'AOISurfaceArea', 'AOINucleiCount',
        'ROICoordinateX', 'ROICoordinateY', 'RawReads', 'AlignedReads',
        'DeduplicatedReads', 'TrimmedReads', 'StitchedReads', 'SequencingSaturation',
        'SequencingSetID', 'UMIQ30', 'RTSQ30', 'GeoMxNgsPipelineVersion',
        'LOT_Human_NGS_Whole_Transcriptome_Atlas_RNA_1_0', 'ROIID', 'SegmentID',
        'ScanWidth', 'ScanHeight', 'ScanOffsetX', 'ScanOffsetY'
    ],
    'CTA': [
        'SlideName', 'ScanLabel', 'ROILabel', 'SegmentLabel', 'SegmentDisplayName',
        'Origin Instrument ID', 'QCFlags', 'AOISurfaceArea', 'AOINucleiCount',
        'ROICoordinateX', 'ROICoordinateY', 'RawReads', 'AlignedReads',
        'DeduplicatedReads', 'TrimmedReads', 'StitchedReads', 'SequencingSaturation',
        'SequencingSetID', 'UMIQ30', 'RTSQ30', 'GeoMxNgsPipelineVersion',
        'LOT_Cancer_Transcriptome_Atlas', 'ROIID', 'SegmentID',
        'ScanWidth', 'ScanHeight', 'ScanOffsetX', 'ScanOffsetY'
    ],
    'MOUSE': [
        'SlideName', 'ScanLabel', 'ROILabel', 'SegmentLabel', 'SegmentDisplayName',
        'Origin Instrument ID', 'QCFlags', 'AOISurfaceArea', 'AOINucleiCount',
        'ROICoordinateX', 'ROICoordinateY', 'RawReads', 'AlignedReads',
        'DeduplicatedReads', 'TrimmedReads', 'StitchedReads', 'SequencingSaturation',
        'SequencingSetID', 'UMIQ30', 'RTSQ30', 'GeoMxNgsPipelineVersion',
        'LOT_Mouse_NGS_Whole_Transcriptome_Atlas_RNA_1_0', 'ROIID', 'SegmentID',
        'ScanWidth', 'ScanHeight', 'ScanOffsetX', 'ScanOffsetY'
    ]
}

class GeomXProcessor:
    def __init__(self, verbose=True):
        self.verbose = verbose
        self.files_data = []
        self.unique_columns_analysis = []
        self.column_mappings = {}
        
    def log(self, message):
        if self.verbose:
            print(f"[INFO] {message}")
    
    def detect_data_type(self, df_columns):
        """데이터 타입 자동 감지 (WTA, CTA, MOUSE)"""
        columns_str = ' '.join(df_columns)
        
        if 'LOT_Cancer_Transcriptome_Atlas' in columns_str:
            return 'CTA'
        elif 'LOT_Mouse_NGS' in columns_str:
            return 'MOUSE'
        else:
            return 'WTA'
    
    def extract_segment_properties(self, file_path):
        """xlsx 파일에서 SegmentProperties 시트 추출"""
        try:
            # openpyxl 엔진 사용하여 안정적으로 읽기
            excel_file = pd.ExcelFile(file_path, engine='openpyxl')
            
            # SegmentProperties 시트 찾기
            sheet_name = None
            for sheet in excel_file.sheet_names:
                if 'SegmentProperties' in sheet:
                    sheet_name = sheet
                    break
            
            if not sheet_name:
                self.log(f"Warning: SegmentProperties sheet not found in {file_path}")
                return None
            
            df = pd.read_excel(excel_file, sheet_name=sheet_name)
            return df
            
        except Exception as e:
            self.log(f"Error reading {file_path}: {str(e)}")
            return None
    
    def analyze_unique_columns(self, file_path):
        """파일의 고유 column 분석"""
        df = self.extract_segment_properties(file_path)
        if df is None:
            return
        
        file_name = Path(file_path).stem
        data_type = self.detect_data_type(df.columns.tolist())
        common_cols = COMMON_COLUMNS.get(data_type, COMMON_COLUMNS['WTA'])
        
        # 고유 columns 찾기
        unique_cols = [col for col in df.columns if col not in common_cols]
        
        self.log(f"File: {file_name}")
        self.log(f"  Data Type: {data_type}")
        self.log(f"  Total Columns: {len(df.columns)}")
        self.log(f"  Unique Columns: {len(unique_cols)}")
        
        # 각 고유 column 분석
        for col in unique_cols:
            # 데이터 타입 추론
            dtype = self.infer_column_type(df[col])
            
            # 고유값 샘플
            unique_values = df[col].dropna().unique()
            value_sample = list(unique_values[:10])  # 최대 10개
            
            self.unique_columns_analysis.append({
                'dataset': file_name,
                'data_type': data_type,
                'column_name': col,
                'dtype': dtype,
                'unique_values': value_sample,
                'unique_count': len(unique_values),
                'null_count': df[col].isna().sum(),
                'total_count': len(df)
            })
        
        # 전체 데이터 저장
        self.files_data.append({
            'file_name': file_name,
            'data_type': data_type,
            'dataframe': df,
            'unique_columns': unique_cols
        })
    
    def infer_column_type(self, series):
        """Column의 데이터 타입 추론"""
        non_null = series.dropna()
        if len(non_null) == 0:
            return 'Empty'
        
        # Boolean 체크
        unique_lower = set(str(v).lower() for v in non_null.unique())
        bool_values = {'true', 'false', 'yes', 'no', '1', '0', 't', 'f', 'y', 'n'}
        if unique_lower.issubset(bool_values):
            return 'Boolean'
        
        # Numeric 체크
        try:
            pd.to_numeric(non_null)
            return 'Numeric'
        except:
            pass
        
        # Category 체크 (고유값이 전체의 10% 미만)
        if len(non_null.unique()) < len(non_null) * 0.1:
            return 'Category'
        
        return 'String'
    
    def process_directory(self, directory_path):
        """디렉토리 내 모든 xlsx 파일 처리"""
        xlsx_files = list(Path(directory_path).glob("*.xlsx"))
        self.log(f"Found {len(xlsx_files)} xlsx files")
        
        for file_path in xlsx_files:
            self.analyze_unique_columns(str(file_path))
    
    def save_analysis_results(self, output_dir):
        """분석 결과 저장"""
        output_path = Path(output_dir)
        output_path.mkdir(exist_ok=True)
        
        # 1. 고유 column 분석 결과
        if self.unique_columns_analysis:
            df_analysis = pd.DataFrame(self.unique_columns_analysis)
            
            # unique_values를 문자열로 변환
            df_analysis['unique_values'] = df_analysis['unique_values'].apply(
                lambda x: ', '.join(map(str, x)) if isinstance(x, list) else str(x)
            )
            
            output_file = output_path / 'unique_columns_analysis.csv'
            df_analysis.to_csv(output_file, index=False)
            self.log(f"Saved unique columns analysis to {output_file}")
            
            # 요약 통계
            print("\n=== Unique Columns Summary ===")
            print(df_analysis.groupby(['data_type', 'column_name']).size().to_string())
    
    def generate_mapping_template(self, output_dir):
        """Column 매핑 템플릿 생성"""
        if not self.unique_columns_analysis:
            self.log("No analysis data available")
            return
        
        df = pd.DataFrame(self.unique_columns_analysis)
        
        # column별 그룹화
        mapping_template = []
        for col_name in df['column_name'].unique():
            col_data = df[df['column_name'] == col_name]
            
            # 모든 고유값 수집
            all_values = set()
            for values_str in col_data['unique_values']:
                if pd.notna(values_str):
                    values = values_str.split(', ') if isinstance(values_str, str) else []
                    all_values.update(values)
            
            mapping_template.append({
                'original_column': col_name,
                'unified_name': col_name,  # 기본값
                'data_types': ', '.join(col_data['data_type'].unique()),
                'datasets': ', '.join(col_data['dataset'].unique()),
                'sample_values': ', '.join(list(all_values)[:20]),
                'value_mapping': '',  # 사용자가 채울 부분
                'notes': ''
            })
        
        df_template = pd.DataFrame(mapping_template)
        output_file = Path(output_dir) / 'column_mapping_template.csv'
        df_template.to_csv(output_file, index=False)
        self.log(f"Saved mapping template to {output_file}")
        
        return df_template
    
    def apply_mappings_and_merge(self, mapping_file, output_dir):
        """매핑 적용 및 metadata 통합"""
        # 매핑 파일 읽기
        df_mapping = pd.read_csv(mapping_file)
        
        # 매핑 딕셔너리 생성
        column_mappings = {}
        value_mappings = {}
        
        for _, row in df_mapping.iterrows():
            original = row['original_column']
            unified = row['unified_name']
            column_mappings[original] = unified
            
            # 값 매핑 파싱
            if pd.notna(row.get('value_mapping', '')):
                mappings = row['value_mapping'].split(',')
                value_map = {}
                for mapping in mappings:
                    if '=' in mapping:
                        old, new = mapping.strip().split('=', 1)
                        value_map[old.strip()] = new.strip()
                if value_map:
                    value_mappings[original] = value_map
        
        # 모든 데이터프레임 통합
        merged_dfs = []
        
        for file_data in self.files_data:
            df = file_data['dataframe'].copy()
            df['_source_file'] = file_data['file_name']
            df['_data_type'] = file_data['data_type']
            
            # Column 이름 매핑
            rename_dict = {}
            for col in df.columns:
                if col in column_mappings:
                    rename_dict[col] = column_mappings[col]
            
            if rename_dict:
                df = df.rename(columns=rename_dict)
            
            # 값 매핑 적용
            for original_col, value_map in value_mappings.items():
                mapped_col = column_mappings.get(original_col, original_col)
                if mapped_col in df.columns:
                    df[mapped_col] = df[mapped_col].replace(value_map)
            
            merged_dfs.append(df)
        
        # 최종 통합
        if merged_dfs:
            merged_metadata = pd.concat(merged_dfs, ignore_index=True, sort=False)
            
            output_file = Path(output_dir) / 'merged_metadata.csv'
            merged_metadata.to_csv(output_file, index=False)
            self.log(f"Saved merged metadata to {output_file}")
            
            print(f"\n=== Merge Summary ===")
            print(f"Total rows: {len(merged_metadata)}")
            print(f"Total columns: {len(merged_metadata.columns)}")
            print(f"Data types: {merged_metadata['_data_type'].value_counts().to_dict()}")
            
            return merged_metadata
        
        return None

class AOIMatcher:
    """Count Matrix와 Metadata 매칭"""
    
    def __init__(self, verbose=True):
        self.verbose = verbose
        
    def log(self, message):
        if self.verbose:
            print(f"[INFO] {message}")
    
    def match_aoi_columns(self, matrix_csv, metadata_csv, output_dir):
        """
        사용자가 이미 처리한 count matrix CSV와 metadata CSV를 매칭
        
        matrix_csv: 첫 column은 gene, 나머지는 AOI columns
        metadata_csv: SegmentDisplayName column이 있는 metadata
        """
        # 파일 읽기
        self.log("Reading count matrix...")
        df_matrix = pd.read_csv(matrix_csv, index_col=0)  # 첫 column을 index로
        matrix_columns = df_matrix.columns.tolist()
        
        self.log("Reading metadata...")
        df_metadata = pd.read_csv(metadata_csv)
        
        if 'SegmentDisplayName' not in df_metadata.columns:
            raise ValueError("Metadata must have 'SegmentDisplayName' column")
        
        # 매칭 수행
        metadata_segments = df_metadata['SegmentDisplayName'].unique()
        
        matched = []
        unmatched_metadata = []
        fuzzy_matched = []
        
        # 매칭 딕셔너리 생성
        match_dict = {}
        
        for segment in metadata_segments:
            if pd.isna(segment):
                continue
                
            segment_str = str(segment)
            
            # 1. 정확 매칭
            if segment_str in matrix_columns:
                matched.append(segment_str)
                match_dict[segment_str] = segment_str
                
            else:
                # 2. Fuzzy 매칭 시도
                found = False
                
                # 파일명_AOI명 형식 찾기
                for col in matrix_columns:
                    # 여러 패턴 시도
                    patterns = [
                        col.endswith(f'_{segment_str}'),
                        col.endswith(f' | {segment_str}'),
                        segment_str in col.split('_')[-1],
                        col.split(' | ')[-1] == segment_str if ' | ' in col else False
                    ]
                    
                    if any(patterns):
                        fuzzy_matched.append((segment_str, col))
                        match_dict[segment_str] = col
                        found = True
                        break
                
                if not found:
                    unmatched_metadata.append(segment_str)
        
        # Matrix에만 있는 columns
        matched_matrix_cols = set(match_dict.values())
        unmatched_matrix = [col for col in matrix_columns if col not in matched_matrix_cols]
        
        # 결과 출력
        print("\n=== Matching Results ===")
        print(f"Exact matches: {len(matched)}")
        print(f"Fuzzy matches: {len(fuzzy_matched)}")
        print(f"Unmatched metadata: {len(unmatched_metadata)}")
        print(f"Unmatched matrix columns: {len(unmatched_matrix)}")
        
        # Fuzzy 매칭 상세
        if fuzzy_matched:
            print("\n=== Fuzzy Matches (Top 10) ===")
            for meta, matrix in fuzzy_matched[:10]:
                print(f"  {meta} → {matrix}")
        
        # 매칭된 metadata 저장
        df_metadata['matrix_column'] = df_metadata['SegmentDisplayName'].map(match_dict)
        df_matched = df_metadata[df_metadata['matrix_column'].notna()]
        
        output_path = Path(output_dir)
        output_path.mkdir(exist_ok=True)
        
        # 매칭된 metadata 저장
        matched_file = output_path / 'matched_metadata.csv'
        df_matched.to_csv(matched_file, index=False)
        self.log(f"Saved matched metadata to {matched_file}")
        
        # 매칭 리포트 저장
        report_file = output_path / 'matching_report.json'
        report = {
            'summary': {
                'total_metadata_segments': len(metadata_segments),
                'exact_matches': len(matched),
                'fuzzy_matches': len(fuzzy_matched),
                'unmatched_metadata': len(unmatched_metadata),
                'unmatched_matrix': len(unmatched_matrix)
            },
            'fuzzy_matches': fuzzy_matched,
            'unmatched_metadata': unmatched_metadata,
            'unmatched_matrix': unmatched_matrix[:50]  # 최대 50개만
        }
        
        with open(report_file, 'w') as f:
            json.dump(report, f, indent=2)
        self.log(f"Saved matching report to {report_file}")
        
        # 매칭된 matrix subset 저장 (옵션)
        if len(df_matched) > 0:
            matched_cols = df_matched['matrix_column'].unique()
            matched_cols = [col for col in matched_cols if col in matrix_columns]
            
            if matched_cols:
                df_matrix_matched = df_matrix[matched_cols]
                matrix_file = output_path / 'matched_count_matrix.csv'
                df_matrix_matched.to_csv(matrix_file)
                self.log(f"Saved matched count matrix to {matrix_file}")
        
        return df_matched, report

def main():
    parser = argparse.ArgumentParser(description='GeomX Data Processor')
    subparsers = parser.add_subparsers(dest='command', help='Commands')
    
    # analyze 명령
    analyze_parser = subparsers.add_parser('analyze', help='Analyze unique columns in xlsx files')
    analyze_parser.add_argument('input_dir', help='Directory containing xlsx files')
    analyze_parser.add_argument('-o', '--output', default='./output', help='Output directory')
    
    # merge 명령
    merge_parser = subparsers.add_parser('merge', help='Apply mappings and merge metadata')
    merge_parser.add_argument('input_dir', help='Directory containing xlsx files')
    merge_parser.add_argument('mapping_file', help='CSV file with column mappings')
    merge_parser.add_argument('-o', '--output', default='./output', help='Output directory')
    
    # match 명령
    match_parser = subparsers.add_parser('match', help='Match count matrix with metadata')
    match_parser.add_argument('matrix_csv', help='Count matrix CSV file')
    match_parser.add_argument('metadata_csv', help='Metadata CSV file')
    match_parser.add_argument('-o', '--output', default='./output', help='Output directory')
    
    args = parser.parse_args()
    
    if args.command == 'analyze':
        processor = GeomXProcessor()
        processor.process_directory(args.input_dir)
        processor.save_analysis_results(args.output)
        processor.generate_mapping_template(args.output)
        
    elif args.command == 'merge':
        processor = GeomXProcessor()
        processor.process_directory(args.input_dir)
        processor.apply_mappings_and_merge(args.mapping_file, args.output)
        
    elif args.command == 'match':
        matcher = AOIMatcher()
        matcher.match_aoi_columns(args.matrix_csv, args.metadata_csv, args.output)
        
    else:
        parser.print_help()

if __name__ == "__main__":
    main()

### ver1. .py 사용부

In [26]:
%%bash
# 2. 고유 column 분석
xlsx_folder="/data/kjc1/projects/#999.GeoMxmeta/pooled/"
echo $xlsx_folder
python geomx_processor.py analyze $xlsx_folder -o ./output

# %%bash를 치면 그 아랫줄은 다 bash로 실행된다.
# %system ls를 치면 한줄씩 실행된다.
# subprocess.run을 사용할 수도 있다.

/data/kjc1/projects/#999.GeoMxmeta/pooled/
[INFO] Found 114 xlsx files
[INFO] Error reading /data/kjc1/projects/#999.GeoMxmeta/pooled/DKD_20240125.xlsx: invalid literal for int() with base 10: 'NaN'
[INFO] Warning: SegmentProperties sheet not found in /data/kjc1/projects/#999.GeoMxmeta/pooled/t4p.xlsx
[INFO] File: #124_mIBD_biologics_95AOI
[INFO]   Data Type: WTA
[INFO]   Total Columns: 30
[INFO]   Unique Columns: 2
[INFO] File: e
[INFO]   Data Type: WTA
[INFO]   Total Columns: 29
[INFO]   Unique Columns: 1
[INFO] File: PC_multi_250219
[INFO]   Data Type: WTA
[INFO]   Total Columns: 33
[INFO]   Unique Columns: 5
[INFO] File: #124_mIBD_bologics_44AOI
[INFO]   Data Type: WTA
[INFO]   Total Columns: 30
[INFO]   Unique Columns: 2
[INFO] File: Kidney TPL_GeoMx_R1
[INFO]   Data Type: WTA
[INFO]   Total Columns: 40
[INFO]   Unique Columns: 13
[INFO] File: #377 Ari_ILC_mo
[INFO]   Data Type: MOUSE
[INFO]   Total Columns: 28
[INFO]   Unique Columns: 0
[INFO] Error reading /data/kjc1/projects/#9

Traceback (most recent call last):
  File "/data/kjc1/mylit/ipynb/geomx_processor.py", line 491, in <module>
    main()
  File "/data/kjc1/mylit/ipynb/geomx_processor.py", line 476, in main
    processor.generate_mapping_template(args.output)
  File "/data/kjc1/mylit/ipynb/geomx_processor.py", line 228, in generate_mapping_template
    if pd.notna(values_str):
ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()


CalledProcessError: Command 'b'# 2. \xea\xb3\xa0\xec\x9c\xa0 column \xeb\xb6\x84\xec\x84\x9d\nxlsx_folder="/data/kjc1/projects/#999.GeoMxmeta/pooled/"\necho $xlsx_folder\npython geomx_processor.py analyze $xlsx_folder -o ./output\n\n# %%bash\xeb\xa5\xbc \xec\xb9\x98\xeb\xa9\xb4 \xea\xb7\xb8 \xec\x95\x84\xeb\x9e\xab\xec\xa4\x84\xec\x9d\x80 \xeb\x8b\xa4 bash\xeb\xa1\x9c \xec\x8b\xa4\xed\x96\x89\xeb\x90\x9c\xeb\x8b\xa4.\n# %system ls\xeb\xa5\xbc \xec\xb9\x98\xeb\xa9\xb4 \xed\x95\x9c\xec\xa4\x84\xec\x94\xa9 \xec\x8b\xa4\xed\x96\x89\xeb\x90\x9c\xeb\x8b\xa4.\n# subprocess.run\xec\x9d\x84 \xec\x82\xac\xec\x9a\xa9\xed\x95\xa0 \xec\x88\x98\xeb\x8f\x84 \xec\x9e\x88\xeb\x8b\xa4.\n'' returned non-zero exit status 1.

In [ ]:
# 3. 매핑 템플릿 편집 (vim, nano 등)
# !vim ./output/column_mapping_template.csv

In [34]:
%%bash
# 4. Metadata 통합
xlsx_folder="/data/kjc1/projects/#999.GeoMxmeta/pooled/"
python geomx_processor.py merge $xlsx_folder ./output/column_mapping_template.csv -o ./output

[INFO] Found 114 xlsx files
[INFO] Error reading /data/kjc1/projects/#999.GeoMxmeta/pooled/DKD_20240125.xlsx: invalid literal for int() with base 10: 'NaN'
[INFO] Warning: SegmentProperties sheet not found in /data/kjc1/projects/#999.GeoMxmeta/pooled/t4p.xlsx
[INFO] File: #124_mIBD_biologics_95AOI
[INFO]   Data Type: WTA
[INFO]   Total Columns: 30
[INFO]   Unique Columns: 2
[INFO] File: e
[INFO]   Data Type: WTA
[INFO]   Total Columns: 29
[INFO]   Unique Columns: 1
[INFO] File: PC_multi_250219
[INFO]   Data Type: WTA
[INFO]   Total Columns: 33
[INFO]   Unique Columns: 5
[INFO] File: #124_mIBD_bologics_44AOI
[INFO]   Data Type: WTA
[INFO]   Total Columns: 30
[INFO]   Unique Columns: 2
[INFO] File: Kidney TPL_GeoMx_R1
[INFO]   Data Type: WTA
[INFO]   Total Columns: 40
[INFO]   Unique Columns: 13
[INFO] File: #377 Ari_ILC_mo
[INFO]   Data Type: MOUSE
[INFO]   Total Columns: 28
[INFO]   Unique Columns: 0
[INFO] Error reading /data/kjc1/projects/#999.GeoMxmeta/pooled/Keloid_LYI_4mm_20240528

Traceback (most recent call last):
  File "/data/kjc1/mylit/ipynb/geomx_processor.py", line 491, in <module>
    main()
  File "/data/kjc1/mylit/ipynb/geomx_processor.py", line 481, in main
    processor.apply_mappings_and_merge(args.mapping_file, args.output)
  File "/data/kjc1/mylit/ipynb/geomx_processor.py", line 301, in apply_mappings_and_merge
    merged_metadata = pd.concat(merged_dfs, ignore_index=True, sort=False)
  File "/home/jaecheon/miniconda3/envs/scenvi/lib/python3.10/site-packages/pandas/core/reshape/concat.py", line 395, in concat
    return op.get_result()
  File "/home/jaecheon/miniconda3/envs/scenvi/lib/python3.10/site-packages/pandas/core/reshape/concat.py", line 680, in get_result
    indexers[ax] = obj_labels.get_indexer(new_labels)
  File "/home/jaecheon/miniconda3/envs/scenvi/lib/python3.10/site-packages/pandas/core/indexes/base.py", line 3885, in get_indexer
    raise InvalidIndexError(self._requires_unique_msg)
pandas.errors.InvalidIndexError: Reindexing only 

CalledProcessError: Command 'b'# 4. Metadata \xed\x86\xb5\xed\x95\xa9\nxlsx_folder="/data/kjc1/projects/#999.GeoMxmeta/pooled/"\npython geomx_processor.py merge $xlsx_folder ./output/column_mapping_template.csv -o ./output\n'' returned non-zero exit status 1.

In [ ]:
# 5. 기존 count matrix와 매칭 (선택사항)
!python geomx_processor.py match your_count_matrix.csv ./output/merged_metadata.csv -o ./final

### ver2. 함수 실행

In [27]:
%%bash
# 2. 고유 column 분석
xlsx_folder="/data/kjc1/projects/#999.GeoMxmeta/pooled/"
echo $xlsx_folder
python geomx_processor_v2.py analyze $xlsx_folder -o ./output

/data/kjc1/projects/#999.GeoMxmeta/pooled/
[INFO] Found 114 xlsx files
[INFO] 
Processing: DKD_20240125.xlsx
[WARNING] Warning: Could not read SegmentProperties from /data/kjc1/projects/#999.GeoMxmeta/pooled/DKD_20240125.xlsx
[INFO] 
Processing: t4p.xlsx
[WARNING] Warning: Could not read SegmentProperties from /data/kjc1/projects/#999.GeoMxmeta/pooled/t4p.xlsx
[INFO] 
Processing: #124_mIBD_biologics_95AOI.xlsx
[INFO] Successfully read /data/kjc1/projects/#999.GeoMxmeta/pooled/#124_mIBD_biologics_95AOI.xlsx using openpyxl
[INFO] File: #124_mIBD_biologics_95AOI
[INFO]   Data Type: WTA
[INFO]   Total Columns: 30
[INFO]   Unique Columns: 2
[INFO]   Ignored Columns: 0
[INFO] 
Processing: e.xlsx
[INFO] Successfully read /data/kjc1/projects/#999.GeoMxmeta/pooled/e.xlsx using openpyxl
[INFO] File: e
[INFO]   Data Type: WTA
[INFO]   Total Columns: 29
[INFO]   Unique Columns: 1
[INFO]   Ignored Columns: 0
[INFO] 
Processing: PC_multi_250219.xlsx
[INFO] Successfully read /data/kjc1/projects/#999.

In [29]:
%%bash
# 2. 결과 확인
cat ./output/unique_columns_analysis.csv | head
cat ./output/column_mapping_template.csv | head

# 3. 문제 파일 확인 (있는 경우)
cat ./output/problematic_files.txt

dataset,data_type,column_name,dtype,unique_values,unique_count,null_count,total_count
#124_mIBD_biologics_95AOI,WTA,Segment 1,Boolean,"True, False",2,0,95
#124_mIBD_biologics_95AOI,WTA,Segment 2,Boolean,"False, True",2,0,95
e,WTA,Segment 1,Boolean,"False, True",2,0,48
PC_multi_250219,WTA,Islet,Boolean,"True, False",2,0,20
PC_multi_250219,WTA,Normal,Boolean,"True, False",2,0,20
PC_multi_250219,WTA,DAPI,Boolean,"False, True",2,0,20
PC_multi_250219,WTA,acini,Boolean,"False, True",2,0,20
#124_mIBD_bologics_44AOI,WTA,Segment 1,Boolean,"True, False",2,0,44
#124_mIBD_bologics_44AOI,WTA,Segment 2,Boolean,"False, True",2,0,44
original_column,unified_name,data_types,datasets,sample_values,value_mapping,notes
Segment 1,Segment 1,"WTA, MOUSE","#124_mIBD_biologics_95AOI, e, #124_mIBD_bologics_44AOI, KSJ2, KSJ, TA3855, 20221228_psoriasis_jsp, bk, 20230103 macrophage , MuOPKidney, yu, Keloid_LYI_2mm_20240528, EoD_analysis_yjbaik, HS_LYI_20240411, HCC_TMA_KMS","[True, False], [False, True]",,
Segment 

In [35]:
%%bash
# 4. Metadata 통합
xlsx_folder="/data/kjc1/projects/#999.GeoMxmeta/pooled/"
python geomx_processor_v2.py merge $xlsx_folder ./output/column_mapping_template.csv -o ./output

[INFO] Found 114 xlsx files
[INFO] 
Processing: DKD_20240125.xlsx
[WARNING] Warning: Could not read SegmentProperties from /data/kjc1/projects/#999.GeoMxmeta/pooled/DKD_20240125.xlsx
[INFO] 
Processing: t4p.xlsx
[WARNING] Warning: Could not read SegmentProperties from /data/kjc1/projects/#999.GeoMxmeta/pooled/t4p.xlsx
[INFO] 
Processing: #124_mIBD_biologics_95AOI.xlsx
[INFO] Successfully read /data/kjc1/projects/#999.GeoMxmeta/pooled/#124_mIBD_biologics_95AOI.xlsx using openpyxl
[INFO] File: #124_mIBD_biologics_95AOI
[INFO]   Data Type: WTA
[INFO]   Total Columns: 30
[INFO]   Unique Columns: 2
[INFO]   Ignored Columns: 0
[INFO] 
Processing: e.xlsx
[INFO] Successfully read /data/kjc1/projects/#999.GeoMxmeta/pooled/e.xlsx using openpyxl
[INFO] File: e
[INFO]   Data Type: WTA
[INFO]   Total Columns: 29
[INFO]   Unique Columns: 1
[INFO]   Ignored Columns: 0
[INFO] 
Processing: PC_multi_250219.xlsx
[INFO] Successfully read /data/kjc1/projects/#999.GeoMxmeta/pooled/PC_multi_250219.xlsx using

Traceback (most recent call last):
  File "/data/kjc1/mylit/ipynb/geomx_processor_v2.py", line 653, in <module>
    main()
  File "/data/kjc1/mylit/ipynb/geomx_processor_v2.py", line 643, in main
    processor.apply_mappings_and_merge(args.mapping_file, args.output)
  File "/data/kjc1/mylit/ipynb/geomx_processor_v2.py", line 460, in apply_mappings_and_merge
    merged_metadata = pd.concat(merged_dfs, ignore_index=True, sort=False)
  File "/home/jaecheon/miniconda3/envs/scenvi/lib/python3.10/site-packages/pandas/core/reshape/concat.py", line 395, in concat
    return op.get_result()
  File "/home/jaecheon/miniconda3/envs/scenvi/lib/python3.10/site-packages/pandas/core/reshape/concat.py", line 680, in get_result
    indexers[ax] = obj_labels.get_indexer(new_labels)
  File "/home/jaecheon/miniconda3/envs/scenvi/lib/python3.10/site-packages/pandas/core/indexes/base.py", line 3885, in get_indexer
    raise InvalidIndexError(self._requires_unique_msg)
pandas.errors.InvalidIndexError: Reindex

CalledProcessError: Command 'b'# 4. Metadata \xed\x86\xb5\xed\x95\xa9\nxlsx_folder="/data/kjc1/projects/#999.GeoMxmeta/pooled/"\npython geomx_processor_v2.py merge $xlsx_folder ./output/column_mapping_template.csv -o ./output\n'' returned non-zero exit status 1.

### ver2. jupyter 실행용

In [ ]:
"""
GeomX Data Processor - Jupyter Notebook 버전
직접 함수를 호출하여 사용할 수 있는 버전
"""

import pandas as pd
import numpy as np
import re
from pathlib import Path
from typing import Dict, List, Optional
import json

# 무시할 column 패턴들
IGNORE_PATTERNS = [
    r'LOT_.*_RNA_\d+$',
    r'Scan_ID_\d+$', 
    r'ROI_ID_\d+$',
    r'^\d+$'
]

def quick_analyze(xlsx_folder: str, output_folder: str = './output'):
    """
    빠른 분석 및 매핑 템플릿 생성
    
    사용 예시:
    >>> quick_analyze('/data/kjc1/projects/#999.GeoMxmeta/pooled/')
    """
    from pathlib import Path
    import warnings
    warnings.filterwarnings('ignore')
    
    # 출력 폴더 생성
    Path(output_folder).mkdir(exist_ok=True)
    
    # xlsx 파일 찾기
    xlsx_files = list(Path(xlsx_folder).glob("*.xlsx"))
    print(f"Found {len(xlsx_files)} xlsx files")
    
    all_unique_cols = []
    problematic_files = []
    
    for file_path in xlsx_files:
        print(f"\nProcessing: {file_path.name}")
        
        try:
            # SegmentProperties 시트 읽기
            df = read_segment_properties_safe(str(file_path))
            
            if df is None:
                problematic_files.append(file_path.name)
                continue
            
            # 고유 column 찾기
            unique_cols = find_meaningful_columns(df)
            
            # 분석 결과 저장
            for col in unique_cols:
                unique_values = df[col].dropna().unique()[:10]
                all_unique_cols.append({
                    'file': file_path.stem,
                    'column': col,
                    'dtype': infer_dtype(df[col]),
                    'samples': ', '.join(map(str, unique_values)),
                    'count': len(df[col].dropna().unique())
                })
                
        except Exception as e:
            print(f"  Error: {str(e)}")
            problematic_files.append(file_path.name)
    
    # 결과 저장
    if all_unique_cols:
        df_result = pd.DataFrame(all_unique_cols)
        
        # 분석 결과 저장
        analysis_file = Path(output_folder) / 'unique_columns_analysis.csv'
        df_result.to_csv(analysis_file, index=False)
        print(f"\nSaved analysis to {analysis_file}")
        
        # 매핑 템플릿 생성
        template = create_mapping_template(df_result)
        template_file = Path(output_folder) / 'column_mapping_template.csv'
        template.to_csv(template_file, index=False)
        print(f"Saved template to {template_file}")
        
        # 요약 출력
        print("\n=== Summary ===")
        print(f"Total unique columns: {df_result['column'].nunique()}")
        print(f"Total files processed: {len(xlsx_files) - len(problematic_files)}")
        
        if problematic_files:
            print(f"\nProblematic files ({len(problematic_files)}):")
            for f in problematic_files[:5]:
                print(f"  - {f}")
            if len(problematic_files) > 5:
                print(f"  ... and {len(problematic_files)-5} more")
    else:
        print("No unique columns found!")
    
    return df_result if all_unique_cols else None


def read_segment_properties_safe(file_path: str) -> Optional[pd.DataFrame]:
    """손상된 xlsx 파일도 안전하게 읽기"""
    import warnings
    warnings.filterwarnings('ignore')
    
    # 여러 방법 시도
    methods = [
        lambda: pd.read_excel(file_path, sheet_name='SegmentProperties', engine='openpyxl'),
        lambda: pd.read_excel(file_path, sheet_name=0, engine='openpyxl'),  # 첫 번째 시트
        lambda: pd.read_excel(file_path, sheet_name='SegmentProperties'),  # 기본 엔진
    ]
    
    for method in methods:
        try:
            df = method()
            
            # 데이터 정리
            if df is not None and not df.empty:
                # NaN column 제거
                df = df.loc[:, df.columns.notna()]
                
                # 숫자 column 이름 정리
                new_cols = []
                for col in df.columns:
                    if isinstance(col, (int, float)):
                        new_cols.append(f"Column_{int(col)}")
                    else:
                        new_cols.append(str(col))
                df.columns = new_cols
                
                return df
        except:
            continue
    
    # 마지막 시도: openpyxl 직접 사용
    try:
        import openpyxl
        wb = openpyxl.load_workbook(file_path, data_only=True, read_only=True)
        
        # 시트 찾기
        sheet_name = None
        for name in wb.sheetnames:
            if 'SegmentProperties' in name or name == wb.sheetnames[0]:
                sheet_name = name
                break
        
        if sheet_name:
            ws = wb[sheet_name]
            data = list(ws.values)
            if data:
                df = pd.DataFrame(data[1:], columns=data[0])
                wb.close()
                return df
        wb.close()
    except:
        pass
    
    return None


def find_meaningful_columns(df: pd.DataFrame) -> List[str]:
    """의미있는 고유 column만 찾기"""
    # 공통 column 패턴
    common_patterns = [
        'SlideName', 'ScanLabel', 'ROILabel', 'SegmentLabel', 'SegmentDisplayName',
        'Origin', 'QCFlags', 'AOISurfaceArea', 'AOINucleiCount',
        'Coordinate', 'Reads', 'Sequencing', 'UMIQ30', 'RTSQ30',
        'GeoMxNgsPipelineVersion', 'ROIID', 'SegmentID',
        'ScanWidth', 'ScanHeight', 'ScanOffset'
    ]
    
    meaningful_cols = []
    
    for col in df.columns:
        if not col:
            continue
            
        col_str = str(col)
        
        # 공통 column인지 확인
        is_common = any(pattern in col_str for pattern in common_patterns)
        if is_common:
            continue
        
        # 무시할 패턴인지 확인
        should_ignore = any(re.match(pattern, col_str) for pattern in IGNORE_PATTERNS)
        if should_ignore:
            continue
        
        # 의미없는 column인지 확인 (모든 값이 같거나, 90% 이상 고유)
        non_null = df[col].dropna()
        if len(non_null) == 0:
            continue
        
        unique_ratio = len(non_null.unique()) / len(non_null)
        if unique_ratio == 1.0 or unique_ratio > 0.9:
            continue
            
        # NULL이 90% 이상인 경우
        if len(non_null) < len(df) * 0.1:
            continue
        
        meaningful_cols.append(col)
    
    return meaningful_cols


def infer_dtype(series: pd.Series) -> str:
    """데이터 타입 추론"""
    non_null = series.dropna()
    if len(non_null) == 0:
        return 'Empty'
    
    # Boolean
    unique_lower = set(str(v).lower() for v in non_null.unique())
    if unique_lower.issubset({'true', 'false', 'yes', 'no', '1', '0'}):
        return 'Boolean'
    
    # Numeric
    try:
        pd.to_numeric(non_null)
        return 'Numeric'
    except:
        pass
    
    # Category (고유값 < 10%)
    if len(non_null.unique()) < len(non_null) * 0.1:
        return 'Category'
    
    return 'String'


def create_mapping_template(df_analysis: pd.DataFrame) -> pd.DataFrame:
    """매핑 템플릿 생성"""
    template = []
    
    for col_name in df_analysis['column'].unique():
        col_data = df_analysis[df_analysis['column'] == col_name]
        
        # 샘플 값 수집
        all_samples = []
        for samples_str in col_data['samples']:
            if samples_str:
                all_samples.extend(str(samples_str).split(', ')[:5])
        
        # 유사한 column 이름 추천 (예: PanCK+, Panck, PANCK → PanCK)
        suggested_name = suggest_unified_name(col_name)
        
        template.append({
            'original_column': col_name,
            'unified_name': suggested_name,
            'appears_in': ', '.join(col_data['file'].unique()[:5]),
            'sample_values': ', '.join(list(set(all_samples))[:10]),
            'value_mapping': '',
            'notes': ''
        })
    
    return pd.DataFrame(template)


def suggest_unified_name(col_name: str) -> str:
    """통일된 column 이름 제안"""
    # 일반적인 패턴들
    patterns = {
        r'(?i)panck?\+?': 'PanCK',
        r'(?i)cd45\+?': 'CD45', 
        r'(?i)ck\+?': 'CK',
        r'(?i)segment[\s_]?(\d+)': r'Segment_\1',
        r'(?i)tumor': 'Tumor',
        r'(?i)stage': 'Stage',
        r'(?i)treatment': 'Treatment'
    }
    
    for pattern, replacement in patterns.items():
        if re.search(pattern, col_name):
            return re.sub(pattern, replacement, col_name, flags=re.IGNORECASE)
    
    return col_name


# Jupyter에서 직접 실행 예시
def demo():
    """
    데모 실행
    Jupyter cell에서:
    >>> from geomx_jupyter import demo
    >>> demo()
    """
    print("GeomX Data Processor - Jupyter Version")
    print("=" * 50)
    print("\n사용 방법:")
    print("1. quick_analyze('/path/to/xlsx/folder/')")
    print("2. 생성된 column_mapping_template.csv 편집")
    print("3. 필요시 merge 및 match 기능 사용")
    print("\n예시:")
    print(">>> result = quick_analyze('/data/kjc1/projects/#999.GeoMxmeta/pooled/')")
    print(">>> result.head()")


if __name__ == "__main__":
    # 직접 실행 시
    import sys
    if len(sys.argv) > 1:
        quick_analyze(sys.argv[1])
    else:
        demo()

In [ ]:
# geomx_jupyter.py 파일 import
from geomx_jupyter import quick_analyze

# 한 줄로 실행
result = quick_analyze('/data/kjc1/projects/#999.GeoMxmeta/pooled/')

# 결과 확인
result.head()

수동 복구

In [ ]:
%%bash
# Excel에서 복구 후 다시 저장하거나
# LibreOffice 사용
libreoffice --headless --convert-to xlsx:"Calc MS Excel 2007 XML" damaged_file.xlsx

수동 복구 - 자동화 (작동은 하는데 복구 제대로 안됨) -> 그냥 손으로 해야할듯.

In [32]:
%%bash
set -euo pipefail

LIST="/data/kjc1/mylit/ipynb/output/problematic_files.txt"
OUTDIR="/data/kjc1/mylit/ipynb/output/fixed"
mkdir -p "$OUTDIR"

# libreoffice(또는 soffice) 선택
CONVERT="libreoffice"
command -v libreoffice >/dev/null 2>&1 || CONVERT="soffice"
echo "[info] using converter: $CONVERT"

# 깨끗한 임시 사용자 프로필 (LO 헤드리스 안정화)
PROFILE_DIR="$(mktemp -d -p /tmp lo-prof-XXXXXX)"
PROFILE_URL="file://$PROFILE_DIR"
echo "[info] using LO profile: $PROFILE_URL"

# outdir 쓰기 테스트
touch "$OUTDIR/.write_test" && rm -f "$OUTDIR/.write_test" || {
  echo "❌ OUTDIR에 쓰기 권한이 없습니다: $OUTDIR" >&2; exit 1; }

while IFS= read -r path; do
  # 헤더/빈줄/주석/비xlsx 스킵
  [[ -z "$path" ]] && continue
  [[ "$path" == Files\ with\ reading\ issues:* ]] && continue
  [[ "$path" =~ ^# ]] && continue
  [[ "${path,,}" != *.xlsx ]] && continue

  if [[ ! -f "$path" ]]; then
    echo "[skip] not a file: $path" >&2
    continue
  fi

  base="$(basename "$path")"
  out_xlsx="$OUTDIR/$base"
  echo "[RUN] $path"

  # A) 자동 필터
  if "$CONVERT" --headless --nologo --nodefault --norestore --nolockcheck \
      -env:UserInstallation="$PROFILE_URL" \
      --convert-to xlsx --outdir "$OUTDIR" "$path"; then
    echo "[OK] auto-filter -> $out_xlsx"; continue
  fi

  # B) 명시 필터
  if "$CONVERT" --headless --nologo --nodefault --norestore --nolockcheck \
      -env:UserInstallation="$PROFILE_URL" \
      --convert-to 'xlsx:"Calc MS Excel 2007 XML"' \
      --outdir "$OUTDIR" "$path"; then
    echo "[OK] explicit-filter -> $out_xlsx"; continue
  fi

  # C) ODS로 우회 후 XLSX 재저장
  tmp_ods="$OUTDIR/${base%.xlsx}.ods"
  if "$CONVERT" --headless --nologo --nodefault --norestore --nolockcheck \
      -env:UserInstallation="$PROFILE_URL" \
      --convert-to ods --outdir "$OUTDIR" "$path"; then
    echo "[OK] to ODS -> $tmp_ods"
    if "$CONVERT" --headless --nologo --nodefault --norestore --nolockcheck \
        -env:UserInstallation="$PROFILE_URL" \
        --convert-to xlsx --outdir "$OUTDIR" "$tmp_ods"; then
      echo "[OK] ODS -> XLSX -> $out_xlsx"
      rm -f "$tmp_ods" || true
      continue
    fi
  fi

  echo "❌ FAIL: $path" >&2
done < "$LIST"

echo "Done. Output -> $OUTDIR"


[info] using converter: libreoffice
[info] using LO profile: file:///tmp/lo-prof-z5djbq
[RUN] /data/kjc1/projects/#999.GeoMxmeta/pooled/DKD_20240125.xlsx


convert /data/kjc1/projects/#999.GeoMxmeta/pooled/DKD_20240125.xlsx -> /data/kjc1/mylit/ipynb/output/fixed/DKD_20240125.xlsx using filter : Calc Office Open XML
[OK] auto-filter -> /data/kjc1/mylit/ipynb/output/fixed/DKD_20240125.xlsx
[RUN] /data/kjc1/projects/#999.GeoMxmeta/pooled/t4p.xlsx


convert /data/kjc1/projects/#999.GeoMxmeta/pooled/t4p.xlsx -> /data/kjc1/mylit/ipynb/output/fixed/t4p.xlsx using filter : Calc Office Open XML
[OK] auto-filter -> /data/kjc1/mylit/ipynb/output/fixed/t4p.xlsx
[RUN] /data/kjc1/projects/#999.GeoMxmeta/pooled/Keloid_LYI_4mm_20240528.xlsx


convert /data/kjc1/projects/#999.GeoMxmeta/pooled/Keloid_LYI_4mm_20240528.xlsx -> /data/kjc1/mylit/ipynb/output/fixed/Keloid_LYI_4mm_20240528.xlsx using filter : Calc Office Open XML
[OK] auto-filter -> /data/kjc1/mylit/ipynb/output/fixed/Keloid_LYI_4mm_20240528.xlsx
[RUN] /data/kjc1/projects/#999.GeoMxmeta/pooled/(empty)BK melanoma.xlsx


convert /data/kjc1/projects/#999.GeoMxmeta/pooled/(empty)BK melanoma.xlsx -> /data/kjc1/mylit/ipynb/output/fixed/(empty)BK melanoma.xlsx using filter : Calc Office Open XML
[OK] auto-filter -> /data/kjc1/mylit/ipynb/output/fixed/(empty)BK melanoma.xlsx
[RUN] /data/kjc1/projects/#999.GeoMxmeta/pooled/CRC(2).xlsx


...

convert /data/kjc1/projects/#999.GeoMxmeta/pooled/HCC_NT_vs_TC_48_KMJ.xlsx -> /data/kjc1/mylit/ipynb/output/fixed/HCC_NT_vs_TC_48_KMJ.xlsx using filter : Calc Office Open XML
[OK] auto-filter -> /data/kjc1/mylit/ipynb/output/fixed/HCC_NT_vs_TC_48_KMJ.xlsx
[RUN] /data/kjc1/projects/#999.GeoMxmeta/pooled/SJ_HY geomx Thymus.xlsx


convert /data/kjc1/projects/#999.GeoMxmeta/pooled/SJ_HY geomx Thymus.xlsx -> /data/kjc1/mylit/ipynb/output/fixed/SJ_HY geomx Thymus.xlsx using filter : Calc Office Open XML
[OK] auto-filter -> /data/kjc1/mylit/ipynb/output/fixed/SJ_HY geomx Thymus.xlsx
[RUN] /data/kjc1/projects/#999.GeoMxmeta/pooled/syringoma re.xlsx


convert /data/kjc1/projects/#999.GeoMxmeta/pooled/syringoma re.xlsx -> /data/kjc1/mylit/ipynb/output/fixed/syringoma re.xlsx using filter : Calc Office Open XML
[OK] auto-filter -> /data/kjc1/mylit/ipynb/output/fixed/syringoma re.xlsx
[RUN] /data/kjc1/projects/#999.GeoMxmeta/pooled/thymus normal.xlsx


convert /data/kjc1/projects/#999.GeoMxmeta/pooled/thymus normal.xlsx -> /data/kjc1/mylit/ipynb/output/fixed/thymus normal.xlsx using filter : Calc Office Open XML
[OK] auto-filter -> /data/kjc1/mylit/ipynb/output/fixed/thymus normal.xlsx
[RUN] /data/kjc1/projects/#999.GeoMxmeta/pooled/thymus240401.xlsx


convert /data/kjc1/projects/#999.GeoMxmeta/pooled/thymus240401.xlsx -> /data/kjc1/mylit/ipynb/output/fixed/thymus240401.xlsx using filter : Calc Office Open XML
[OK] auto-filter -> /data/kjc1/mylit/ipynb/output/fixed/thymus240401.xlsx
Done. Output -> /data/kjc1/mylit/ipynb/output/fixed


## Metadata Merger

### ver2.